In [1]:
# Titanic Survival Prediction
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Load dataset
df = pd.read_csv("/content/archive (5).zip")

# Feature engineering
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['Title'] = df['Name'].str.extract(r',\s*([^\.]+)\.')
df['Cabin_known'] = df['Cabin'].notnull().astype(int)

# Select features
features = ['Pclass','Sex','Age','Fare','Embarked','FamilySize','Title','Cabin_known']
target = 'Survived'
X = df[features]
y = df[target]
X['Embarked'].fillna('S', inplace=True)

# Preprocessing
num_feats = ['Age','Fare','FamilySize']
cat_feats = ['Pclass','Sex','Embarked','Title','Cabin_known']

num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('ohe', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer([('num', num_pipe, num_feats), ('cat', cat_pipe, cat_feats)])

# Model
clf = Pipeline([('pre', preprocessor), ('rf', RandomForestClassifier(n_estimators=200, random_state=42))])

# Train-test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Save model
joblib.dump(clf, "titanic_model.pkl")


/tmp/ipython-input-4140156344.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['Embarked'].fillna('S', inplace=True)
/tmp/ipython-input-4140156344.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Embarked'].fillna('S', inplace=True)


Accuracy: 0.7932960893854749
[[93 17]
 [20 49]]
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       110
           1       0.74      0.71      0.73        69

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



['titanic_model.pkl']